# ASSIGNMENT 3 
### NoSQL MongoDB

## Abstract

The aim of this assignment is to convert the SQL Database into NoSQL Database .
The doman of the last assignment was supermarket dataset. The data set was taken from kaggle and had information about the customer transaction for products purchased from a particular Product Line.
Twitter was the social media scraped for tweets about the product line. This assignment converts the previous SQL Database into NoSQL database and stores everything in MongoDB Database.

## Data Source

##### The Link for the data source we have used for this assignment  and the previous assignment- 
#####  https://www.kaggle.com/aungpyaeap/supermarket-sales 

##### What is SQL Database?
SQL are relational databases. They use SQL queries to manipulate the data. Although SQL is a very powwerfull language, it is extremely restrictive. It requires predefined schemas to determine the structure of your data before you start working with it.
SQL requires all the data to follow the same structure for working with it.
Changing the structure of the schema can be disruptive for the whole system.
SQL is veryically scalable. SQL Databases are table based and Follow ACID properties, - Availability, consistency, Isolation and Durability.
Some popular SQL Databases are - PostgreSQL, MySQL, Oracle, Microsoft SQL Server

##### What is NoSQL Database?
NoSQL is primarily called the Non-Relational database or distributed databases. NoSQL has dynamic schemas for non structured data storage. In NoSQL data can be stored in various ways, document-oriented, graph based, column oriented or key value store.
NoSQL database is horizantally scalable , which means it manages traffic by data sharding or adding more servers to your NoSQL Database.
NoSQL databases follow the Brewers CAP theorem ( Consistency, Availability and Partition Tolerance)
Some popoular NoSQL Database are - Redis, RavenDB, Cassandra, MongoDB, CouchDB


## Data Auditing

Data Auditing is performed on the dataset from kaggle for checking for its validity ,completeness and consistency

In [4]:
from pymongo import MongoClient
import sqlite3
import json
import os
from twitter import *
import tweepy 
import pandas as pd
import pprint
import datetime

### Reading the CSV File for Sales data

In [5]:
supermarket_data = pd.read_csv("supermarket.csv")
supermarket_data.head()

,Invoice ID,first_name,last_name,address,phone1,phone2,email,Branch,City,Customer type,...,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,CITY,STATE
0,750-67-8428,James,Butt,6649 N Blue Gum St,504-621-8927,504-845-1427,jbutt@gmail.com,A,Yangon,Member,...,548.9715,1/5/19,13:08,Ewallet,522.83,4.761905,26.1415,9.1,New Orleans,LA
1,226-31-3081,Josephine,Darakjy,4 B Blue Ridge Blvd,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,C,Naypyitaw,Normal,...,80.2200,3/8/19,10:29,Cash,76.40,4.761905,3.8200,9.6,Brighton,MI
2,631-41-3108,Art,Venere,8 W Cerritos Ave #54,856-636-8749,856-264-4130,art@venere.org,A,Yangon,Normal,...,340.5255,3/3/19,13:23,Credit card,324.31,4.761905,16.2155,7.4,Bridgeport,NJ
3,123-19-1176,Lenna,Paprocki,639 Main St,907-385-4412,907-921-2010,lpaprocki@hotmail.com,A,Yangon,Member,...,489.0480,1/27/19,20:33,Ewallet,465.76,4.761905,23.2880,8.4,Anchorage,AK
4,373-73-7910,Donette,Foller,34 Center St,513-570-1893,513-549-4561,donette.foller@cox.net,A,Yangon,Normal,...,634.3785,2/8/19,10:37,Ewallet,604.17,4.761905,30.2085,5.3,Hamilton,OH


### Information about supermarket dataset

In [6]:
supermarket_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
Invoice ID                 1000 non-null object
first_name                 1000 non-null object
last_name                  1000 non-null object
address                    1000 non-null object
phone1                     1000 non-null object
phone2                     1000 non-null object
email                      1000 non-null object
Branch                     1000 non-null object
City                       1000 non-null object
Customer type              1000 non-null object
Gender                     1000 non-null object
Product line               1000 non-null object
Unit price                 1000 non-null float64
Quantity                   1000 non-null int64
Tax 5%                     1000 non-null float64
Total                      1000 non-null float64
Date                       1000 non-null object
Time                       1000 non-null object
Payment                    1

### Checking for missing values in the data set

In [7]:
supermarket_data.isnull().any()

Invoice ID                 False
first_name                 False
last_name                  False
address                    False
phone1                     False
phone2                     False
email                      False
Branch                     False
City                       False
Customer type              False
Gender                     False
Product line               False
Unit price                 False
Quantity                   False
Tax 5%                     False
Total                      False
Date                       False
Time                       False
Payment                    False
cogs                       False
gross margin percentage    False
gross income               False
Rating                     False
CITY                       False
STATE                      False
dtype: bool

### Describing the Schema

In [8]:
supermarket_data.describe()

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1.000000e+03,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905e+00,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,6.220360e-14,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905e+00,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905e+00,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905e+00,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905e+00,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905e+00,49.650000,10.00000


## Installing the Python Drivers

In [9]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install twitter

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


## Twitter API

In [12]:
consumer_Key = "44qTzVYo9lBvDDrt7JY2jUc9V"
consumer_Secret_Key = "zINUwbwlRzSguW30I4cR28QH9j0248S5BmRsDVKXlAGgZoFk11"
access_token = "1246250838265335810-HLZHUQ2lfxIJ40OZnEGMBf1fOjy0p5"
access_token_secret = "4OerR3m6TQPleqXUtlvCzzqc1crMXWPKIzupyXRs3C4dn"

auth = tweepy.OAuthHandler(consumer_Key, consumer_Secret_Key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [13]:
productLines=set(supermarket_data["Product line"])

## Scraping The tweets for ProductLines

In [14]:
productLinesDictionary = {}
def productLineSearch(query):
    tweets = []
    count = 100
    
    searchedTweets = api.search(q=query,count=count,since="2020-04-08", until="2020-04-09")
    for tweet in searchedTweets:
        tags = []
        hashtags = (tweet._json["entities"]["hashtags"])
        screenname = tweet._json["user"]['screen_name']
        for tag in hashtags:
            tags.append(tag["text"])
        if len(tags) == 0:
            tags = ""
        tweet_obj ={
                "product_Line":query,
                "created_at":tweet.created_at,
                "screenname":screenname,
                "id":tweet.id,
                "text":tweet.text,
                "retweet":tweet.retweet_count,
                "favCount":tweet.favorite_count,
                "tags":tags
        }
        tweets.append(tweet_obj)
    productLinesDictionary[query] = tweets

In [15]:
# Finds the tweets for all product lines
for product in productLines:
    productLineSearch(product)

## Printing all the tweets scraped for all the ProductLines 

In [16]:
productLinesDictionary

{'Food and beverages': [{'product_Line': 'Food and beverages',
   'created_at': datetime.datetime(2020, 4, 8, 22, 40, 52),
   'screenname': 'VIRBOY21',
   'id': 1248018000122281984,
   'text': 'RT @Shell_Canada: Our priority is the health &amp; safety of everyone affected by this pandemic, but we want to do more. In the coming weeks, a…',
   'retweet': 678,
   'favCount': 0,
   'tags': ''},
  {'product_Line': 'Food and beverages',
   'created_at': datetime.datetime(2020, 4, 8, 21, 49, 26),
   'screenname': 'RayDubicki',
   'id': 1248005053928992768,
   'text': 'TIL there is an extensive section of the Seattle City Code devoted to milk.\n\nAnd it’s probably against the law for… https://t.co/OCkLCFLinS',
   'retweet': 0,
   'favCount': 1,
   'tags': ''},
  {'product_Line': 'Food and beverages',
   'created_at': datetime.datetime(2020, 4, 8, 21, 47, 25),
   'screenname': 'HofNorthwoods',
   'id': 1248004547814903810,
   'text': '@stealthygeek Golfing! Our local course where I have great m

### Breaking The dataset into tables

### Branch Table

In [17]:
#Branch Main Table
Branch = pd.DataFrame()
Branch[['Branch_Id','City']] = supermarket_data[['Branch','City']]
Branch = Branch.drop_duplicates()
Branch = Branch.reset_index(drop = True)
Branch.head()

,Branch_Id,City
0,A,Yangon
1,C,Naypyitaw
2,B,Mandalay


### Product Table

In [18]:
#Product 
Product = pd.DataFrame()
Product[['Invoice ID','Product_Line','Unit_Price','Branch_Id']] = supermarket_data[['Invoice ID','Product line','Unit price','Branch']]
Product.index.name = "Product_Id"
Product.head()

,Invoice ID,Product_Line,Unit_Price,Branch_Id
Product_Id,,,,
0,750-67-8428,Health and beauty,74.69,A
1,226-31-3081,Electronic accessories,15.28,C
2,631-41-3108,Home and lifestyle,46.33,A
3,123-19-1176,Health and beauty,58.22,A
4,373-73-7910,Sports and travel,86.31,A


### ProductLine Table

In [19]:
#Product Line
ProductLine = pd.DataFrame()
ProductLine[['Product_Line']] = supermarket_data[['Product line']]
ProductLine = ProductLine.drop_duplicates()
ProductLine = ProductLine.reset_index(drop = True)
ProductLine.head()

,Product_Line
0,Health and beauty
1,Electronic accessories
2,Home and lifestyle
3,Sports and travel
4,Food and beverages


### Ratings Table

In [20]:
#Ratings
Ratings = pd.DataFrame()
Ratings[['Product_Line','Customer_Rating','Branch_Id']] = supermarket_data[['Product line','Unit price','Branch']]
Ratings.index.name ="Ratings_Id"
Ratings.head()

,Product_Line,Customer_Rating,Branch_Id
Ratings_Id,,,
0,Health and beauty,74.69,A
1,Electronic accessories,15.28,C
2,Home and lifestyle,46.33,A
3,Health and beauty,58.22,A
4,Sports and travel,86.31,A


### Customer Table

In [21]:
#Customer
Customer = pd.DataFrame()
#Customer[['First_Name','Last_name','PhoneNumber_1','PhoneNumber_2','Email','Gender','Customer_Type','Amount_Billed','Address','State','City']] = supermarket_data[['first_name','last_name','phone1','phone2','email','Gender','Customer type','Total','Address','STATE','CITY']]

Customer[['Invoice_Id','First_Name','Last_name','PhoneNumber_1','PhoneNumber_2','Email','Gender','Customer_Type','Amount_Billed','Address','STATE','CITY','Product_Line']] = supermarket_data[['Invoice ID','first_name','last_name','phone1','phone2','email','Gender','Customer type','Total','address','STATE','CITY','Product line']]
Customer.index.name ="Customer_Id"
Customer.head()

,Invoice_Id,First_Name,Last_name,PhoneNumber_1,PhoneNumber_2,Email,Gender,Customer_Type,Amount_Billed,Address,STATE,CITY,Product_Line
Customer_Id,,,,,,,,,,,,,
0,750-67-8428,James,Butt,504-621-8927,504-845-1427,jbutt@gmail.com,Female,Member,548.9715,6649 N Blue Gum St,LA,New Orleans,Health and beauty
1,226-31-3081,Josephine,Darakjy,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org,Female,Normal,80.2200,4 B Blue Ridge Blvd,MI,Brighton,Electronic accessories
2,631-41-3108,Art,Venere,856-636-8749,856-264-4130,art@venere.org,Male,Normal,340.5255,8 W Cerritos Ave #54,NJ,Bridgeport,Home and lifestyle
3,123-19-1176,Lenna,Paprocki,907-385-4412,907-921-2010,lpaprocki@hotmail.com,Male,Member,489.0480,639 Main St,AK,Anchorage,Health and beauty
4,373-73-7910,Donette,Foller,513-570-1893,513-549-4561,donette.foller@cox.net,Male,Normal,634.3785,34 Center St,OH,Hamilton,Sports and travel


### Payment Table

In [22]:
#Payment
Payment = pd.DataFrame()
Payment[['Invoice_Id','Total-Amount','Date_Of_Purchase','Time','Payment_Type']] = supermarket_data[['Invoice ID','Total','Date','Time','Payment']]
Payment.head()

,Invoice_Id,Total-Amount,Date_Of_Purchase,Time,Payment_Type
0,750-67-8428,548.9715,1/5/19,13:08,Ewallet
1,226-31-3081,80.2200,3/8/19,10:29,Cash
2,631-41-3108,340.5255,3/3/19,13:23,Credit card
3,123-19-1176,489.0480,1/27/19,20:33,Ewallet
4,373-73-7910,634.3785,2/8/19,10:37,Ewallet


### Tax Table

In [23]:
#Tax
Tax = pd.DataFrame()
Tax[['Tax','Payment_Invoice']] = supermarket_data[['Tax 5%','Invoice ID']]
Tax.head()

,Tax,Payment_Invoice
0,26.1415,750-67-8428
1,3.8200,226-31-3081
2,16.2155,631-41-3108
3,23.2880,123-19-1176
4,30.2085,373-73-7910


### State Table

In [24]:
#State
State = pd.DataFrame()
State[['State']] = supermarket_data[['STATE']]
State = State.drop_duplicates()
State = State.reset_index(drop = True)
State.index.name = "State Id"
State.head()

,State
State Id,
0,LA
1,MI
2,NJ
3,AK
4,OH


### Cost Of Goods Table

In [25]:
#Cost of Goods Sold
COG = pd.DataFrame()
COG[['Total_Billed_Amount']] = supermarket_data[['Total']]
COG.head()

,Total_Billed_Amount
0,548.9715
1,80.2200
2,340.5255
3,489.0480
4,634.3785


### Gross Details Table

In [26]:
#Gross Details
Gross_Details = pd.DataFrame()
#Gross_Details[['Cost_Of_Goods_Sold','Gross_Income','Gross_Margine_Percentage','Payment_Invoice_Id']] = supermarket_data[['cogs','gross income','gross margine percentage','Invoice ID']]
Gross_Details[['Cost_Of_Goods_Sold','Gross_Income','Gross_Margin_Percentage','Payment_Invoice_Id']] = supermarket_data[['cogs','gross income','gross margin percentage','Invoice ID']]
Gross_Details.index.name = "Gross_Details_Id"
Gross_Details.head()

,Cost_Of_Goods_Sold,Gross_Income,Gross_Margin_Percentage,Payment_Invoice_Id
Gross_Details_Id,,,,
0,522.83,26.1415,4.761905,750-67-8428
1,76.40,3.8200,4.761905,226-31-3081
2,324.31,16.2155,4.761905,631-41-3108
3,465.76,23.2880,4.761905,123-19-1176
4,604.17,30.2085,4.761905,373-73-7910


# NoSQL Design explaination

We want to store the Data in a mongoDB database. In mongoDB, data is stored in the form of collections and documents. MongoDb stores the data in the form of Key value pairs.

##### MongoDB
MongoDB is an opensource document database. It is written in C++. MongoDB is cross platformm and provides high perfomance, high availability and easy scalability. MongoDB works on the concept of collection and document.
MongoDB doesnt use table and rows to store data but instead it uses collection of JSON like documents. Documents store embeded fields so that related data can be stored within them. It is schemaless so be do not need to specift the column for the database befor einserting document withing them.

##### Collections
It is a collection of documents and is equivalent to tables in RDBMS table. A collection exists within a single database.
documents within the collection can have documents with different fields however mostly all the documents in the collection are related to eachother. 

##### Documents
Documents have a set of key value pairs. Documents have Dynamic schema which means documents in the same schema do not have same fields or structure. Different fields in the collection document may hold different values. 




##### Design Decisions
The database is designed by considering the productline at the top level of the hierarchy. The ProductLine Database stores documents of every ProductLine. Each of these documents store information about all the customers who have purchased soemthing in the productline category and the tweets that are related to the productLine.

####  ProductLineDoc 
This collection has documents that are segerated according to the productlines.
The structure of each of these documents is - 
   ###### ProductLine: Name of the ProductLine
   ###### customerDictionary :Information about the customers,
   ###### ProductLine_Rating :ProductLine rating by customers
   ###### ProductTweet :Tweets associated with the productline
   
#### CustomerDictionary
This disctionary stores information about the customers who have purchased soemthing in the productLine caregory-
This distionary also contains payment details about the customers.
Things included in this dictionary are - 
'First_Name','Last_name','PhoneNumber_1','PhoneNumber_2','Email','Gender','Customer_Type','Amount_Billed','Address','STATE','CITY','Product_Line','Invoice_Id','Total_amount','Date_Of_Purchase','Time','Payment_Type'

#### ProductTweets
This dictionary stores the tweets that are related to the productline. This dictionary also stores all the hashtags that are associated with the tweet.
Information stored in this dictionary -
"product_Line","created_at","screenname","id","text","retweet","favCount","tags"


In [27]:
ProductLineDoc = []
ProductLine_Dict = {}
ProductLineSalesDict = {}
customerDict = {}
TwitterPosts = {}
ProductLine_rating ={}
productTweet = []
Customer_Ratings = []
#Customer[['First_Name','Last_name','PhoneNumber_1','PhoneNumber_2','Email','Gender','Customer_Type','Amount_Billed','Address','STATE','CITY','Product_Line']] = supermarket_data[['first_name','last_name','phone1','phone2','email','Gender','Customer type','Total','address','STATE','CITY','Product line']]
FinalDictionary = []

for i in range(len(ProductLine)):
    for c in range(len(Customer)):
        #Store customer details and the details of their payments in JSON 
        if Customer["Product_Line"][c] == ProductLine["Product_Line"][i]:
            customerDict = {
                "FirstName":Customer["First_Name"][c],
                "LastName":Customer["Last_name"][c],
                "Address":Customer["Address"][c],
                "PhoneNumber1":Customer["PhoneNumber_1"][c],
                "PhoneNumber2":Customer["PhoneNumber_2"][c],
                "email":Customer["Email"][c],
                "gender":Customer["Gender"][c],
                "customer_type":Customer["Customer_Type"][c],
                "amount_billed":Customer["Amount_Billed"][c],
                "state":Customer["STATE"][c],
                "city":Customer["CITY"][c],
            }
            for  p in range(len(Payment)):
                if Payment["Invoice_Id"][p] == Customer["Invoice_Id"][c]:
                    customerDict["Total_amount"] = Payment["Total-Amount"][p]
                    customerDict["Date_Of_Purchase"] = Payment["Date_Of_Purchase"][p]
                    customerDict["Time"] = Payment["Time"][p]
                    customerDict["Payment_Type"] = Payment["Payment_Type"][p]
                    
    #Store the ratings for each of the Product Lines                
    for r in range(len(Ratings)):
        if Ratings["Product_Line"][r] == ProductLine["Product_Line"][i]:
            Customer_Ratings.append(Ratings["Customer_Rating"][r])
            
    productTweet = productLinesDictionary[ProductLine["Product_Line"][i]]
    mainDict = {
            "ProductLine":ProductLine["Product_Line"][i],
            "customerDictionary":customerDict,
            "ProductLine_Rating":ProductLine_rating,
            "productTweet":productTweet
             }
    ProductLineDoc.append(mainDict)

### The Size of the ProductLine Document

In [28]:
len(ProductLineDoc)

6

### Storing the collection created in MongoDB

In [29]:
client = MongoClient('localhost', 27017)

In [30]:
print(client.list_database_names())

['MEANStackDB', 'MoviesDB', 'PleaseDB', 'ProductLine_Database', 'UserdataDB', 'admin', 'config', 'customers', 'local', 'mylib', 'nodeauth', 'userdb']


### Deleting the database if it already exists

In [31]:
client["ProductLine_Database"]['ProductLine'].drop()

### Checking if the database is deleted

In [32]:
print(client.list_database_names())

['MEANStackDB', 'MoviesDB', 'PleaseDB', 'UserdataDB', 'admin', 'config', 'customers', 'local', 'mylib', 'nodeauth', 'userdb']


### Creating the database -> ProductLine_Database and the database collection ->ProductLine

In [33]:
mydatabase = client["ProductLine_Database"]
mycol = mydatabase["ProductLine"]
x = mycol.insert_many(ProductLineDoc)
print(client.list_database_names())

['MEANStackDB', 'MoviesDB', 'PleaseDB', 'ProductLine_Database', 'UserdataDB', 'admin', 'config', 'customers', 'local', 'mylib', 'nodeauth', 'userdb']


### Retriving all the data stored in MongoDB

In [34]:
for pl in mycol.find({}):
    pprint.pprint(pl)

{'ProductLine': 'Health and beauty',
 'ProductLine_Rating': {},
 '_id': ObjectId('5e8e5532b5101d7d130493c7'),
 'customerDictionary': {'Address': '9166 Devon St #905',
                        'Date_Of_Purchase': '1/29/19',
                        'FirstName': 'Avery',
                        'LastName': 'Veit',
                        'Payment_Type': 'Ewallet',
                        'PhoneNumber1': '01748-625058',
                        'PhoneNumber2': '01369-185737',
                        'Time': '13:46',
                        'Total_amount': 42.3675,
                        'amount_billed': 42.3675,
                        'city': 'Boise',
                        'customer_type': 'Normal',
                        'email': 'avery@veit.co.uk',
                        'gender': 'Male',
                        'state': 'ID'},
 'productTweet': [{'created_at': datetime.datetime(2020, 4, 8, 22, 47, 58),
                   'favCount': 0,
                   'id': 1248019784039411712,
  

                   'product_Line': 'Home and lifestyle',
                   'retweet': 103,
                   'screenname': 'JessSargus',
                   'tags': '',
                   'text': 'RT @latimes: Gov. Gavin Newsom said the order to '
                           'stay at home to help slow the spread of the '
                           'coronavirus will continue “until further notic…'},
                  {'created_at': datetime.datetime(2020, 4, 8, 22, 13, 22),
                   'favCount': 0,
                   'id': 1248011078253993989,
                   'product_Line': 'Home and lifestyle',
                   'retweet': 103,
                   'screenname': 'nkdec',
                   'tags': '',
                   'text': 'RT @latimes: Gov. Gavin Newsom said the order to '
                           'stay at home to help slow the spread of the '
                           'coronavirus will continue “until further notic…'},
                  {'created_at': datetime.dat

                   'favCount': 0,
                   'id': 1247938780968800257,
                   'product_Line': 'Sports and travel',
                   'retweet': 0,
                   'screenname': 'gordwright',
                   'tags': '',
                   'text': '@llikemoyd Donate those “deposit” bottles guys. '
                           'Sports teams will need to fundraise soon (and by '
                           'the sounds of it t… https://t.co/VTHcFRmhKC'},
                  {'created_at': datetime.datetime(2020, 4, 8, 17, 22, 1),
                   'favCount': 0,
                   'id': 1247937756728262661,
                   'product_Line': 'Sports and travel',
                   'retweet': 1,
                   'screenname': 'TheShergar',
                   'tags': '',
                   'text': 'RT @FCurran17: Despite all the doom &amp; gloom '
                           'and its gonna get worse in new week or two deaths '
                           'wise I fully 

                   'text': 'RT @TOO_LIVE_MILZ: If you or someone you know '
                           'doesn’t have access to FOOD &amp; BEVERAGES during '
                           'this pandemic and are in BROOKLYN let me know…'},
                  {'created_at': datetime.datetime(2020, 4, 8, 18, 14, 35),
                   'favCount': 0,
                   'id': 1247950984493428737,
                   'product_Line': 'Food and beverages',
                   'retweet': 0,
                   'screenname': 'bashdash4',
                   'tags': '',
                   'text': '15 days until I forego all food and beverages in '
                           'the name of the Lord'},
                  {'created_at': datetime.datetime(2020, 4, 8, 18, 11, 49),
                   'favCount': 0,
                   'id': 1247950291242278913,
                   'product_Line': 'Food and beverages',
                   'retweet': 29,
                   'screenname': 'xShaunieex',
                 

                   'tags': '',
                   'text': 'Fashion, Beauty, Home Accessories &amp; Baby '
                           'centrepointstores\n'
                           '\n'
                           'Buy gift cards and use them online with ease \n'
                           'orders wi… https://t.co/Dy1Lmsqa98'},
                  {'created_at': datetime.datetime(2020, 4, 8, 22, 30, 45),
                   'favCount': 0,
                   'id': 1248015450882990081,
                   'product_Line': 'Fashion accessories',
                   'retweet': 0,
                   'screenname': 'cudaeducation',
                   'tags': ['fashionbusiness'],
                   'text': '2020 FASHION BIZ REPORT:  Get real statistics, '
                           'facts and information about the #fashionbusiness '
                           '|… https://t.co/sMgHg3PBZV'},
                  {'created_at': datetime.datetime(2020, 4, 8, 22, 30, 21),
                   'favCount': 0,
 

### Answering all the Database questions for the assignment - 


### Q1> - What are tags are associated with a person, place or thing?

In [35]:
def getHashTags():
    tweetCount = set()
    for i in range(len(ProductLineDoc)):
        productLine = mycol.find({})[i]['ProductLine']
        print(productLine)
        ProductLineTweets = mycol.find({})[i]['productTweet']
        for twt in ProductLineTweets:
            if len(twt['tags']) != 0:
                for t in twt['tags']:
                    tweetCount.add(t)
        print(tweetCount)
        tweetCount.clear()
        print()
        print()
getHashTags()

Health and beauty
{'health', 'Health', 'home', 'coronavirus', 'immunityboost', 'wellness', 'writingcommunity', 'COVID19', 'watch', 'covid19', 'exercise', 'Microplastics', 'beauty', 'gadget', 'clothing', 'sleep', 'TextureHunterGatherer', 'Woonsocket', 'Diet'}


Electronic accessories
{'hashtag3', 'phone_chargersSpring', 'electronic_accessories', 'phone_cables', 'Gaming', 'phone_glass', 'montblanc', 'phone_case'}


Home and lifestyle
{'coronavirus', 'disinfect', 'clean'}


Sports and travel
{'EQRoundtable', 'business', 'club', 'CoronaOutbreak', 'UK', 'sports', 'COVID19', 'religious', 'vacations', 'Travel', 'travels', 'travel', 'China', 'coronavirus'}


Food and beverages
{'beverages', 'contactfree', 'long', 'Biotechnology', 'Doneraile', 'Nigeria', 'SNAP', 'FEC', 'tobacco', 'foodindustry', 'stock', 'food', 'TheFive', 'Liberia', 'team', 'BritishColumbia'}


Fashion accessories
{'influencer', 'classy', 'scarifications', 'model', 'beautiful', 'look', 'fashionbusiness', 'style', 'jewelry', 'b

### Q2>-  What social media users are like other social media users in your domain?

In [36]:
def similarSocialMediaUsers(productLine, user):
    similarUsers = set()
    tweets = mycol.find_one({'ProductLine':productLine})['productTweet']
    for tweet in tweets:
        if tweet["screenname"] == user:
            tags = tweet['tags']
            break
    print(tags)
    for tag in tags:
        for tweet in tweets:
            if tag in tweet['tags']:
                similarUsers.add(tweet['screenname'])
    print(similarUsers)

In [47]:
similarSocialMediaUsers("Health and beauty","gd959098")

['writingcommunity']
{'ccthewriter1', 'gd959098', 'KatinaParon', 'weischoice'}


### Q3>  What people, places or things are popular in your domain?

In [38]:
#Tweet Count within a time range
trending ={}
def trendingDomain(productLine):
    startDate = datetime.datetime(2020, 4, 8, 8, 0, 0)
    endDate =   datetime.datetime(2020, 4, 8, 9, 0, 0)
    numbTweets = 0
    retweets = 0;
    tweets = mycol.find_one({'ProductLine':productLine})['productTweet']
    for tweet in tweets:
        if (tweet['created_at'] > startDate) and (tweet['created_at'] > endDate):
            numbTweets=numbTweets+1
            retweets = retweets+tweet['retweet']
    trend = numbTweets+retweets
    trending[productLine] = trend

In [39]:
for product in productLines:
    trendingDomain(product)

In [40]:
sortedTrendDictionary = sorted(trending.items(), key=lambda x: x[1], reverse=True)  

In [41]:
print(trending)

{'Food and beverages': 13235, 'Electronic accessories': 101, 'Sports and travel': 571, 'Home and lifestyle': 7746, 'Fashion accessories': 416, 'Health and beauty': 613}


In [42]:
print(sortedTrendDictionary)

[('Food and beverages', 13235), ('Home and lifestyle', 7746), ('Health and beauty', 613), ('Sports and travel', 571), ('Fashion accessories', 416), ('Electronic accessories', 101)]


#### The Most Popular productLine is -

In [43]:
sortedTrendDictionary[0]

('Food and beverages', 13235)

### UserName and the tags associated with that user

In [44]:
tweets = mycol.find_one({'ProductLine':"Health and beauty"})['productTweet']
for tweet in tweets:
    print(tweet['screenname'],tweet['tags'])

GrayskyX 
velocipedus 
mvsiii71 
Treuburg 
JackStewartCham 
dawn27331807 
TheVibeDealer3 
HollyThomas61 ['covid19', 'immunityboost', 'home']
EmanHabbash 
ewhoma_ 
USIIV 
BidorBuy247 
Its_3One4 
Ali_yeganeh_s 
Gentleman__7 
TraceyBenmore 
InStyle 
4ChrissyO 
tea_natasha 
Raul38557638 
AALawnLandscape 
Beauty_Rushhhh 
ah_marachy 
tania_hanyu ['Health']
_ataei 
ReGenFriends 
tmj_rip_adv ['Woonsocket']
kaelynforde 
ccthewriter1 ['writingcommunity']
gd959098 ['writingcommunity']
Crisp_Mag 
Carols_Garcia 
CeciliaDaForno1 
BoBAUall 
KatinaParon ['writingcommunity']
Greenhouses1 
ElleCanada 
ArnicareUSA ['Diet', 'exercise', 'sleep']
honkide 
weischoice ['writingcommunity']
tomwlsn31 ['COVID19']
BidorBuy247 
PRJournoRequest 
Shahinaz6 
ladyboarder9669 ['covid19', 'immunityboost', 'home']
se23_tweets ['TextureHunterGatherer']
JournalistJill 
seuoceancleanup ['Microplastics']
Honeysblood 
lynnettepeck 
LizAtkin ['TextureHunterGatherer']
buynbuyshop ['health', 'clothing', 'gadget', 'beauty', 'watc

### Q4> What people, places or things are popular in your domain?

In [51]:
def popularInDomain(productLine):
    print("The domain is -----> "+productLine)
    popularity = {}
    tweets = mycol.find_one({'ProductLine':productLine})['productTweet']
    for tweet in tweets:
        for tag in tweet['tags']:
            if tag in popularity:
                popularity[tag] = popularity[tag]+1
            else:
                popularity[tag] = 1
                
    sortedDictionary = sorted(popularity.items(), key=lambda x: x[1], reverse=True)    
    #Printing top 5 in every domain -
    if len(popularity)<5:
        rg = len(popularity)
    else:
        rg = 5
    for x in range(0,rg):
        print(sortedDictionary[x][0])


In [52]:
for i in range(len(ProductLineDoc)):
    productLine = mycol.find({})[i]['ProductLine']
    popularInDomain(productLine)

The domain is -----> Health and beauty
writingcommunity
covid19
immunityboost
home
TextureHunterGatherer
The domain is -----> Electronic accessories
montblanc
electronic_accessories
phone_case
phone_glass
phone_cables
The domain is -----> Home and lifestyle
clean
disinfect
coronavirus
The domain is -----> Sports and travel
vacations
business
sports
religious
club
The domain is -----> Food and beverages
foodindustry
Doneraile
contactfree
TheFive
SNAP
The domain is -----> Fashion accessories
friends
Giveaway
fashion
style
accessories


### Report

###### Sample Data -
The data source for this assignment is - https://www.kaggle.com/aungpyaeap/supermarket-sales The source of data that we have used is super store dataset. The data set is taken from Kaggle.

###### Aim
The aim of this assignment was to convert the RDBMS created in the last assignment into NoSQL DBMS. Then answer te questions for the databse.

##### Data Auditing
The data retrived from the kaggle datasource was audited to ensure that the data is all clean, consistent and valid. auditing was done to ensure that there are no null values or duplicate entries in the data source

After t=auditing was done, the data was broken into different tables according to the RDBMS requirements

##### Design Decision Explaination
The design decision explaination have been explained in a very detailed format in the the above section please refer to the design explaination section for more details about this

##### Database Questions 
###### Question 1 - What are tags are associated with a person, place or thing?
The tags have been scraped for each tweet. A set has been used to store all the tags that are related to the productline so that there would be no duplicates in the hash tags retrived

###### Question 2 - What social media users are like other social media users in your domain?
The social media that use same hashtags are similar to other social media users. This is the logic used for finding the users that are similar to each other.

###### Question 3 - What people, places or things are popular in your domain?
The productLines which have maximum tweets and retweets in an hour of a specified date are most popular in my domain.

###### Question 4 - What people, places or things are trending in your domain? (A trend is popularity over time.)
The hashtags which hace been posted the maximum number of times are the things that ateh people are most talking about and are most popular in that domain.


#####  Professionalism
Professionalism has been maintained throughout this assignment 

### Conclusion

The SQL Relational database was converted into NoSQL database - MongoDB. Everything was stored in the form of documents and collection in mySQL database. 
As per the the requirements of this assignmnets, all the 4 question have been answered.
The data is audited and cleaned before using for analysis.
The Database design decisions have been clearly explained
A discrete report has been created for explaianing the work in the assignment.
We have tested that the we can retrive the data for different usecases. We have taken exta efforts to make sure that professionalism has been maintained in the assignment.

### Contributions

Our contributed : 30%

External source (Blogs, wikipedia, sample assignment and youtube videos): 30%

Provided by the professor : 40%

### Citations

https://www.geeksforgeeks.org/difference-between-sql-and-nosql/

https://www.tutorialspoint.com/mongodb/index.htm

https://www.sitepoint.com/an-introduction-to-mongodb/

https://github.com/nikbearbrown/INFO_6210/tree/master/Project/Jobs_DB_Project/IPYNB
